# Computing Statistical Estimate on Probabilistic Safety with DeepBayes

Probabilistic Safety for BNNs: $Prob_{\theta \sim p(\theta | \mathcal{D})}\big( f^{\theta}(x') \in S \quad \forall x' \in T \big)$

In this notebook, we go over how to compute statistical estimates of probabilistic safety for BNNs with DeepBayes such that we have control over the error and confidence of our estimate. 

#### Example notebook takes 10 sections to run in total
(Times are reported for an M1 Pro Macbook)

In [1]:
import os
import sys
import time
import logging
import numpy as np
import deepbayes
from deepbayes import PosteriorModel
from deepbayes.analyzers import IBP_prob
from deepbayes.analyzers import IBP_upper
from deepbayes.analyzers import FGSM
from deepbayes.analyzers import massart_bound_check
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

#### Load in the MNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float64").reshape(-1, 28*28)
X_test = X_test.astype("float64").reshape(-1, 28* 28)

#### Define safe and unsafe predicates

These functions will take in the input upper and lower bounds as well as the values of the output 
logits and then will need to return True if the output is within the safe region i.e., $f^{\theta}(x') \in S$

In [3]:
def predicate_safe(iml, imu, ol, ou):
    v1 = tf.one_hot(TRUE_VALUE, depth=10)
    v2 = 1 - tf.one_hot(TRUE_VALUE, depth=10)
    v1 = tf.squeeze(v1); v2 = tf.squeeze(v2)
    worst_case = tf.math.add(tf.math.multiply(v2, ou), tf.math.multiply(v1, ol))
    if(np.argmax(worst_case) == TRUE_VALUE):
        return True
    else:
        return False
    
def predicate_worst(worst_case):
    if(np.argmax(worst_case) != TRUE_VALUE):
        return False
    else:
        return True

#### Load in the pretrained BNN Model

In [4]:
bayes_model = PosteriorModel("PosteriorModels/VOGN_MNIST_Posterior/")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 128)            100480    
_________________________________________________________________
dense_1 (Dense)              (None, 1, 10)             1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
BayesKeras detected the above model 
 None


#### Set Verification Parameters and compute Decision Lower Bound

##### Parameters: 
* Index - The index of the test set input we want to estimate the robustness of
* Epsilon - The size of the input set that we consider for verification
* Confidence - The probability that our estimate falls outside of the error range
* Delta - The specified error range


In [5]:
INDEX = 0
EPSILON = 0.025
CONFIDENCE = 0.75
DELTA = 0.25

img = np.asarray([X_test[INDEX]])
TRUE_VALUE = y_test[INDEX]
img = np.asarray([X_test[INDEX]])
img_upper = np.clip(np.asarray([X_test[INDEX]+(EPSILON)]), 0, 1)
img_lower = np.clip(np.asarray([X_test[INDEX]-(EPSILON)]), 0, 1)

In [6]:
start = time.process_time()
p_safe_attack, iterations_attack, mean = massart_bound_check(bayes_model, img, EPSILON, predicate_worst, cls=TRUE_VALUE,
                                                             confidence=CONFIDENCE, delta=DELTA, alpha=0.05, classification=True,
                                                             verify=False, chernoff=False, verbose=True)
attk_time = time.process_time() - start
d_safe_attack = predicate_worst(mean)

BayesKeras. Maximum sample bound = 17
Working on iteration: 1.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 2.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 3.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 4.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 5.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 6.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 7.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 8.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 9.0 	 Bound: 17 	 Param: 1.0
Working on iteration: 10.0 	 Bound: 16 	 Param: 1.0
Working on iteration: 11.0 	 Bound: 16 	 Param: 1.0
Working on iteration: 12.0 	 Bound: 15 	 Param: 1.0
Working on iteration: 13.0 	 Bound: 15 	 Param: 1.0
Working on iteration: 14.0 	 Bound: 14 	 Param: 1.0
Exited becuase 15.0 >= 14
Mean is returned as zero because massart does not provide valid bounds on the mean.


In [7]:
print("The BNN and input has statistical robustness %s for epsilon 0.025"%(p_safe_attack))
print("Statistical check with confidence 0.75 and delta 0.25 too %s iterations and %s seconds"%(iterations_attack, attk_time))

The BNN and input has statistical robustness 1.0 for epsilon 0.025
Statistical check with confidence 0.75 and delta 0.25 too 15.0 iterations and 4.320111000000001 seconds
